In [4]:
import torch
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn
from PIL import Image
import os
import numpy as np

# Define class names
class_names = ['CityA', 'CityB', 'CityC', 'CityD']

# Specify input folders for each city
folders = {
    'CityA': '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/ma-boston/buildings',
    'CityB': '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/nc-charlotte/buildings',
    'CityC': '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/ny-manhattan/buildings',
    'CityD': '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/pa-pittsburgh/buildings'
}

# Define a custom dataset class
class CityDataset(Dataset):
    def __init__(self, folders, transform=None):
        self.image_paths = []
        self.labels = []
        self.transform = transform
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(folders.keys())}

        for class_name, folder in folders.items():
            for filename in os.listdir(folder):
                if filename.endswith(('.jpg', '.jpeg', '.png')):
                    self.image_paths.append(os.path.join(folder, filename))
                    self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations: resize, convert to tensor, normalize
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create dataset
dataset = CityDataset(folders, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=False)

# Load a pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Modify the final layer to match the number of classes
num_classes = len(class_names)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Save model outputs as features
def extract_and_save_features(model, data_loader, file_name='city_features.npy'):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for images, label in data_loader:
            images = images.to(device)
            outputs = model(images)
            features.append(outputs.cpu().numpy())
            labels.append(label.numpy())
    
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    np.save(file_name, {'features': features, 'labels': labels})

# Extract and save features
extract_and_save_features(model, data_loader, 'city_features.npy')

/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


OSError: [Errno 89] Operation canceled

In [ ]:
def train_and_save_model(model, train_loader, num_epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

    # Save the trained model weights
    torch.save(model.state_dict(), 'resnet50_city_classifier.pth')

# Train and save the model
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

train_and_save_model(model, train_loader)

In [ ]:
import numpy as np

# Load saved model weights
model.load_state_dict(torch.load('resnet50_city_classifier.pth'))

# Function to classify a new image
def classify_new_image(image_path, model, transform):
    model.eval()
    input_image = Image.open(image_path)
    input_tensor = transform(input_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        probabilities = F.softmax(output, dim=1)
        probabilities = probabilities.cpu().numpy().flatten()

    predictions = [(class_names[i], prob * 100) for i, prob in enumerate(probabilities)]
    predictions.sort(key=lambda x: x[1], reverse=True)
    return predictions

# Example of classifying a new image
new_image_path = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/ny-brooklyn/buildings/buildings_1370.jpg'
predictions = classify_new_image(new_image_path, model, transform)
for label, percentage in predictions:
    print(f'Predicted class: {label}, Confidence: {percentage:.2f}%')